In [ ]:
# Structural Concrete according to Eurocode 2

This project has a set of tools to deal with Eurocode 2 design rules
Public at [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/pcachim/concrete/master?filepath=ec2-base.ipynb)

## Material properties

Concrete

- [Basic properties](./ec2-material.ipynb)
- [Creep](./ec2-fluencia.ipynb)
- Shrinkage

Steel

- Reinforcing steel
- Prestressing steel

## Structural elements

- [beams](./ec2-vigas.ipynb)

## JSON database file

A [database](./eurocodes.json) containing information regarding the properties of Eurocode materials, loads and sections.

In [34]:
import math
import json
import pandas as pd

In [35]:
f = open("eurocodes.json", "r")
database = json.loads(f.read())
concs = database["Eurocodes"]["Materials"]["Concrete"]
reinfs = database["Eurocodes"]["Materials"]["Reinforcement"]
prestr = database["Eurocodes"]["Materials"]["Prestress"]

In [36]:
ec2db = pd.DataFrame.from_dict(concs["Classes"])
ec2db

,C20/25,C25/30,C30/37,C35/45,C40/50,C45/45,C50/55,C55/67,C60/75,C70/85
fck,20.0,25.0,30.0,35.0,40.0,45.0,50.0,55.00,60.0,70.00
fcm,28.0,33.0,38.0,43.0,48.0,53.0,58.0,63.00,68.0,78.00
fctm,2.2,2.6,2.9,3.2,3.5,3.8,4.1,4.20,4.4,4.60
fctk05,1.5,1.8,2.0,2.2,2.5,2.7,2.9,3.00,3.1,3.20
Ecm,30000.0,31000.0,33000.0,34000.0,35000.0,36000.0,37000.0,38000.00,39000.0,41000.00
epsc2,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.20,2.3,2.40
epscu2,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.10,2.9,2.70
n,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.75,1.6,1.45


In [37]:
ec2db = pd.DataFrame.from_dict(reinfs["Classes"])
ec2db

,A400,A500,S400,S500
fyk,400.0,500.0,400.0,500.0
Es,200000.0,200000.0,200000.0,200000.0


In [38]:
b = 0.3
h = 1.0
A = b*h
I = b*h**3/12
print (A, I)

0.3 0.024999999999999998


## Dados geométricos

In [39]:
A = 0.3
I = 0.025
Zg  = 0.5
H = 1.0
ap = 0.15
Wi = I/Zg
Ws = I/(H-Zg)

In [40]:
conc_class_0 = "C25/30"
conc_class_inf = "C35/45"

## Dados ações

In [41]:
Med = 500
Ned = 0 # Tração ≥ 0, compressão ≤ 0
Po = 1700 # Sempre positivo

In [42]:
Med_inf = 1000
Ned_inf = 0
P_inf = 1800

## Dados pré-esforço

In [43]:
conc = concs["Classes"][conc_class_0]
conc_inf = concs["Classes"][conc_class_inf]
ep = Zg-ap

## Resultados

In [44]:
Sig_inf = -Po/A+Ned/A-Po*ep/Wi+Med/Wi
Sig_sup = -Po/A+Ned/A+Po*ep/Ws-Med/Ws
print (Sig_inf, Sig_sup)

-7566.666666666668 -3766.666666666667


In [45]:
fck = conc["fck"]
fctm = conc["fctm"]
Ecm = conc["Ecm"]
print (fck, fctm, Ecm)
s1 = "OK" if Sig_sup <= 1000.0*fctm else "KO"
s2 = "OK" if Sig_inf >= -600.0*fck else "KO"
print (s1, s2)

25.0 2.6 31000.0
OK OK


In [46]:
Sig_inf_inf = -P_inf/A+Ned_inf/A-P_inf*ep/Wi+Med_inf/Wi
Sig_sup_inf = -P_inf/A+Ned_inf/A+P_inf*ep/Ws-Med_inf/Ws
print (Sig_inf_inf, Sig_sup_inf)

1400.0 -13400.0


In [47]:
fck = conc_inf["fck"]
fctm = conc_inf["fctm"]
Ecm = conc_inf["Ecm"]
print (fck, fctm, Ecm)
s1 = "OK" if Sig_inf_inf <= 1000.0*fctm else "KO"
s2 = "OK" if Sig_sup_inf >= -600.0*fck else "KO"
print (s1, s2)

35.0 3.2 34000.0
OK OK
